### Rolling Linear Regression (no leakage)
### *“Simplicity is the ultimate sophistication.”*
<div style="text-align: center"><b><i>Leonardo da Vinci</i></b></div>

In [ ]:
import numpy as np
import pandas as pd
import jpx_tokyo_market_prediction

train_path = "../input/jpx-tokyo-stock-exchange-prediction/train_files/"
usecols = ["Date","SecuritiesCode","Close"]
df = pd.read_csv(train_path+"stock_prices.csv", usecols=usecols)
df = df.sort_values(["SecuritiesCode","Date"], ascending=[True,True]).ffill()
df = pd.pivot_table(df,values="Close",index="Date",columns="SecuritiesCode")
df = df[-1:] # using only the last day from train set

env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()

for (prices, _, _, _, _, sample_prediction) in iter_test:
    tr = pd.pivot_table(prices,values="Close",index="Date",columns="SecuritiesCode")
    df = pd.concat([df, tr]).ffill()
    y = df[-2:].values # Get two last close values
    pred = 1-y[1]/(y[1]+(y[1]-y[0])) # Calc Slope Linear with last two points
    sample_prediction['Rank'] = np.argsort(np.argsort(pred))[::-1]
    env.predict(sample_prediction)